# Service configuration

This page provides an overview of the options you can use to configure the services defined in docker compose. In fact, this is a description of how to set a specific property on the containers created by docker compose.

Check the [corresponding page](https://docs.docker.com/reference/compose-file/services/) of the official documentation.

## Project name

Containers run with Compose have special properties, such as the **project name**. Compose uses a project name to isolate environments from each other. For more information, see the ["Specifying a project name"](https://docs.docker.com/compose/how-tos/project-name/) page in the official documentation. By default, Compose uses the folder of the Compose file as the project name, but you can specify the `--project-folder` or `--project-name` parameters to change the container's project.

To check which project a container belongs to, look for the path `.[0].Config.Labels["com.docker.compose.project"]` in the container's inspect result.

---

The following cell defines the docker compose file in the `temp_direcotry` and executes it.

In [ ]:
mkdir temp_folder
cat << EOF > temp_folder/temp_compose.yml
services:
  linux1:
    image: alpine
    container_name: project_example
EOF
docker compose -f temp_folder/temp_compose.yml up &> /dev/null

Now let's check the `{{index .Config.Labels "com.docker.compose.project"}}` of the deployed container's configuration.

In [ ]:
docker inspect -f '{{index .Config.Labels "com.docker.compose.project"}}' project_example

temp_folder


The project name is `temp_folder`, which is the folder where `compose.yml` is located.

In [ ]:
docker compose -f temp_folder/temp_compose.yml down &> /dev/null
rm -r temp_folder

## Environment variables

You can set environment variables for a service under the `environment` key. 

---

This example defines an interactive, detached service in a Docker Compose file and sets an environment variable `ENV_VARIABLE`.

In [ ]:
cat << EOF > docker-compose.yml
services:
  linux1:
    image: alpine
    container_name: compose_example
    environment:
      - ENV_VARIABLE=message from container
    stdin_open: true
    tty: true
EOF

docker compose up -d &> /dev/null

Let's check if it contains variables just like we specified. 

In [ ]:
docker exec compose_example sh -c "echo \$ENV_VARIABLE"

message from container


Stop compose after all.

In [ ]:
docker compose down &> /dev/null
rm docker-compose.yml

## Network

For each coimpose project by default is created special default network, all containers that belongs to this project will automaticly connected to this network.

---

The following cell starts a simple compose project.

In [ ]:
cat << EOF > /tmp/compose.yml
services:
  linux1:
    image: alpine
    container_name: network_example
EOF

docker compose -f /tmp/compose.yml up &> /dev/null

Now there is a docker network that corresponds to this project: compose is run in the `tmp` directory, so it inherits it's name, as the result network created for this project will have the name `temp_deafult`. 

In [ ]:
docker network ls | grep tmp

708aec9a9313   tmp_default   bridge    local


The following cell prints the network to which the `network_example` container belongs.

In [ ]:
docker inspect -f '{{range $key, $value := .NetworkSettings.Networks}}{{$key}} {{end}}' network_example

tmp_default 


It's a network created for the compose project.

In [ ]:
docker compose -f /tmp/compose.yml down &> /dev/null

## Command

You can specify commands that would be used to run the container by using the `command` field il the service settings - the container in question would use command to run itself.

---

The following cell shows `compose.yml` that uses `command` directive, runs corresponding container and shows the result.

In [ ]:
cat << EOF > /tmp/compose.yml
services:
  service:
    image: alpine
    command: echo "hello world"
EOF

docker compose -f /tmp/compose.yml run --rm service

hello world


## Service from dockerfile

Sometimes it's useful to configure a service to start from an image that hasn't been built yet—just by specifying its dockerfile. Such a configuration can be defined using the `build` key inside the service definition.

---

The following cell creates a "toy project" consisting of a "dockerfile" and a "compose.yml" that represents how the corresponding service should be deployed.

In [41]:
mkdir /tmp/serv_dockerfile &> /dev/null
cd /tmp/serv_dockerfile

In [42]:
cat << EOF > dockerfile
FROM python:3.12
CMD ["python3", "-c", "print(\"I'm a custom container\")"]
EOF

The simpliest configuration of the `build` key is used here. Just `build .` means that the service will be started from the docker file located in the same directory as `compose.yml`.

In [43]:
cat << EOF > compose.yml
services:
  service:
    build: .
EOF

The following cell executes this configuration.

In [52]:
docker-compose up 2>&1 | grep '^service-1\s*|'

service-1  | I'm a custom container


As a result, there is a message in the logs of the `service-1` that is specified in the dockerfile.